In [2]:
# import das libs necessárias
import pandas as pd # trabalhar com dataframes
import numpy as np # realizacao de algumas operacoes com matrizes

#imagens
import cv2 # transformacoes faceis em imagens
from PIL import Image # trabalhar com imagens

# ferramentas
import glob # exploracao de diretorios
from pylab import *
import tqdm
import importlib
import time

# plot 
import matplotlib.pyplot as plt # plotagem
%matplotlib inline

# Machine Learning
from sklearn.neural_network import MLPClassifier 
from sklearn.model_selection import GridSearchCV # Grid Search para rodar todos os parametros

#importando ferramentas já implementadas anteriormente
import sampling #retorna uma amostra para treino
importlib.reload(sampling)
import PCA_and_Inverse
importlib.reload(PCA_and_Inverse)
import feature_selection
importlib.reload(feature_selection)

<module 'feature_selection' from 'E:\\Reconhecimento de Padrões\\pattern_recognition\\MLP\\feature_selection.py'>

## Cria uma amostra das imagens do tamanho definido e carrega em dataframes

In [3]:
#carregando amostra dos dados
X, Y = sampling.get_sample(400)

100%|████████████████████████████████████████████████████████████████████████████████| 400/400 [00:06<00:00, 64.39it/s]


## Extração de características

- Calcula as Componentes principais para os dados carregados

In [3]:
pca = PCA_and_Inverse.PCA_and_Inverse()
pca_components = pca.get_PCA(X=X, n_components=64)

## Seleção de características

- Calcula as melhores features a serem utilizadas pelos métodos de:
    - Informação Mútua
    - Algoritmo Genético

#### Informação Mútua

In [ ]:
mic_threshold = 0.05
mic_features_df = feature_selection.get_MIC_features(X=X, Y=Y)
mic_features = list(mic_features_df[mic_features_df[0] > mic_threshold].index)

c:\users\theus\appdata\local\programs\python\python37\lib\site-packages\sklearn\utils\validation.py:752: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


#### Algoritmo Genético

In [ ]:
ga_features = feature_selection.get_GA_features(X=X, Y=Y)
ga_features

## Gera as combinações de parâmetros para rodar o SVM

In [8]:
mlp_classifier = MLPClassifier(solver="sgd", learning_rate="constant")
parameters = [{"hidden_layer_sizes":[(2,), (5,), (10,), (25,)], "learning_rate_init":[0.1, 0.5, 0.9]}]

#### Roda todas as combinações da SVM para as features

In [9]:
grid_search_full = GridSearchCV(mlp_classifier, parameters, verbose=5, n_jobs=-1, cv=3)

In [10]:
grid_search_full.fit(X, Y)

Fitting 3 folds for each of 12 candidates, totalling 36 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  10 tasks      | elapsed:   47.7s
[Parallel(n_jobs=-1)]: Done  36 out of  36 | elapsed:  3.2min finished
c:\users\theus\appdata\local\programs\python\python37\lib\site-packages\sklearn\neural_network\multilayer_perceptron.py:916: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


GridSearchCV(cv=3, error_score='raise-deprecating',
       estimator=MLPClassifier(activation='relu', alpha=0.0001, batch_size='auto', beta_1=0.9,
       beta_2=0.999, early_stopping=False, epsilon=1e-08,
       hidden_layer_sizes=(100,), learning_rate='constant',
       learning_rate_init=0.001, max_iter=200, momentum=0.9,
       n_iter_no_change=10, nesterovs_momentum=True, power_t=0.5,
       random_state=None, shuffle=True, solver='sgd', tol=0.0001,
       validation_fraction=0.1, verbose=False, warm_start=False),
       fit_params=None, iid='warn', n_jobs=-1,
       param_grid=[{'hidden_layer_sizes': [(2,), (5,), (10,), (25,)], 'learning_rate_init': [0.1, 0.5, 0.9]}],
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=5)

In [ ]:
grid_search_full.cv_results_

#### Roda todas as combinações da SVM para as features extraídas pelo PCA

In [ ]:
grid_search_pca = GridSearchCV(svm_classifier, parameters, verbose=5, n_jobs=-1, cv=3)

In [ ]:
grid_search_pca.fit(pca_components, Y)

In [ ]:
grid_search_pca.cv_results_

#### Roda todas as combinações da SVM para as features selecionadas pela Informação Mútua (MIC)

In [ ]:
grid_search_mic = GridSearchCV(svm_classifier, parameters, verbose=5, n_jobs=-1, cv=3)

In [ ]:
grid_search_mic.fit(X[mic_features], Y)

In [ ]:
grid_search_mic.cv_results_

#### Roda todas as combinações da SVM para as features selecionadas pelo Algoritmo Genético (GA)

In [ ]:
grid_search_ga = GridSearchCV(svm_classifier, parameters, verbose=5, n_jobs=-1, cv=3)

In [ ]:
grid_search_ga.fit(pca_components, Y)

In [ ]:
grid_search_ga.cv_results_